# Image Edge Detection
This kernel applies a compass edge detection filter on a grayscale image using sobel operators on 8 directions.

Each pixel in the input or output images is packed in a 32bit word.

## Kernel Code
The OpenCL code of your kernel can be edited here. It will be atomatically saved to the file named in the first line.

In [2]:
%%writefile OpenCL_code/compass_edge_detection.cl
#include "FGPUlib.c"

__kernel void compass_edge_detection(__global unsigned *in, __global unsigned *out)
{
  unsigned x = get_global_id(1);
  unsigned y = get_global_id(0);
  unsigned rowLen = get_global_size(0);


  // return on boarder pixels
  bool border =  x < 1 | y < 1 | (x>rowLen-2) | (y>rowLen-2);
  if(border) 
    return;
  //read pixels
  unsigned p[3][3];
  unsigned p00 = in[(x-1)*rowLen+y-1];
  unsigned p01 = in[(x-1)*rowLen+y];
  unsigned p02 = in[(x-1)*rowLen+y+1];
  unsigned p10 = in[x*rowLen+y-1];
  unsigned p11 = in[x*rowLen+y];
  unsigned p12 = in[x*rowLen+y+1];
  unsigned p20 = in[(x+1)*rowLen+y-1];
  unsigned p21 = in[(x+1)*rowLen+y];
  unsigned p22 = in[(x+1)*rowLen+y+1];
  int G[8] = {0};
  //find edges in 4 directions
  G[0] =  -1*p00 +0*p01 +1*p02 +
          -2*p10 +0*p11 +2*p12 +
          -1*p20 +0*p21 +1*p22;
  G[1] =  -2*p00 -1*p01 +0*p02 +
          -1*p10 +0*p11 +1*p12 +
          -0*p20 +1*p21 +2*p22;
  G[2] =  -1*p00 -2*p01 -1*p02 +
          -0*p10 +0*p11 +0*p12 +
          +1*p20 +2*p21 +1*p22;
  G[3] =  -0*p00 -1*p01 -2*p02 +
          +1*p10 +0*p11 -1*p12 +
          +2*p20 +1*p21 +0*p22;
  //compute the edges in the remaining 4 directions by inversion
  G[4] = -G[0];
  G[5] = -G[1];
  G[6] = -G[2];
  G[7] = -G[3];
  //taking the maximum value on all directions
  int max_val = G[0], i;
  for(i = 1; i < 8; i++)
    max_val = G[i] < max_val ? max_val:G[i];
  //write the result to output
  out[x*rowLen+y] = max_val;
}


Overwriting ../OpenCL_code/compass_edge_detection.cl


## Create Objects

In [3]:
import sys
sys.path.append("../FGPU/FGPU")
from FGPU import FGPU

fgpu= FGPU()

## Program Hardware

In [4]:
fgpu.set_bitFile("../bitstreams/PYNQ/1CU_fp_100MHz.bit")
fgpu.download_bitstream()

## Compile Kernel

#### compile_kernels parameters: 
0 (true/false) -> output the logs (print compiled code as objdump)

1 (true/false) -> floating point support

In [6]:
fgpu.set_kernel_file("OpenCL_code/compass_edge_detection.cl")
fgpu.compile_kernel(False, False)

Compiling /home/xilinx/jupyter_notebooks/FGPU/notebooks/OpenCL_code/compass_edge_detection.cl
Compiling succeeded!




## Read Input Image & Allocate Memory

In [7]:
from pynq import allocate
import scipy.misc
import sys
from cffi import FFI
import numpy as np
ffi = FFI()

#read input image
img=scipy.misc.face()
nRows, nCols = img.shape[0:2]; # get image dimensions
length = nRows*nCols # length of input and output array

# Allocate continuous memory for kernel parameters
img_ptr = allocate(shape=(length,), dtype='int')
edges_fgpu = allocate(shape=(length,), dtype='int')

# copy the input image to CMA region
if type(img) == np.ndarray:
    ffi.memmove(img_ptr, img, img.size*img.dtype.itemsize) 
else:
    AssertionError()

## Configure Kernel

In [8]:
# bind allocated memory to kenel parameters
fgpu.set_paramerter(0, img_ptr)
fgpu.set_paramerter(1, edges_fgpu)
# setup index space
fgpu.set_num_dimensions(2)
fgpu.set_size(size=nCols, dim=0)
fgpu.set_size(size=nRows, dim=1)
fgpu.set_work_group_size(wg_size=8, dim=0)
fgpu.set_work_group_size(wg_size=8, dim=1)
fgpu.set_offset(value=0, dim=0)
fgpu.set_offset(value=0, dim=1)

## Execute On FGPU

In [9]:
#download kernel binary code and settings to hardware
fgpu.download_kernel()
#execute and wait until finish
execTime = fgpu.execute_kernel()
print ("Execution time =", int(execTime*1000), "ms")

Execution time = 805 ms


## Compute without FGPU
Here the same task will be executed in python 

In [67]:
img=scipy.misc.face()

import numpy as np
import scipy.misc 
import scipy.ndimage
import time
from matplotlib import pyplot as plt

def detect_edges(image,masks):
    # This function filters "image" with all filters in "masks" and returns the maxilim result for each pixel
    edges = abs(scipy.ndimage.convolve(image,masks[0]))
    for mask in masks[1:4]:
        edges=np.maximum(abs(scipy.ndimage.convolve(image,mask)), edges)
    return edges

# Sobel operators for edge detection
Sobel=[
        [
        [-1,+0,+1],
        [-2,+0,+2],
        [-1,+0,+1]
        ],
        [
        [-2,-1,+0],
        [-1,+0,+1],
        [+0,+1,+2]
        ],
        [
        [-1,-2,-1],
        [+0,+0,+0],
        [+1,+2,+1]
        ],
        [
        [-0,-1,-2],
        [+1,+0,-1],
        [+2,+1,+0]
        ] 
]

#compute task
start = time.time()
#img = img[:,:,:,None]
Sobel = numpy.array(Sobel)
Sobel = Sobel[:,:,:,None]
edges=detect_edges(img, Sobel)
finalOutput = edges.squeeze()
end = time.time()
print ("Execution time =", int((end-start)*1000), "ms")


#plot input and output images
#plt.figure(1, figsize=(10, 10))
#plt.subplot(121),plt.imshow(img, cmap='Greys_r'),plt.title('Original')
#plt.xticks([]), plt.yticks([])
#plt.subplot(122),plt.imshow(edges, cmap='Greys_r'),plt.title('Edges')
#plt.xticks([]), plt.yticks([])
#plt.show()

Execution time = 2288 ms
